In [1]:
import numpy as np
%matplotlib inline
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt
import math
import networkx as nx


In [2]:
% cd ..

/Users/cedoz/Documents/Stanford/CS224W/Network-Analysis-of-Human-Protein-Interaction-Network


In [3]:
from read_graph import read_graph
from common.pipeline import Pipeline
from common.feature_generators import *
from validation import *

In [4]:
cancer = get_cancer()
mendelian = get_mendelian()
drugbank_target_all = get_drugbank(molecule_type="target", subset="all")

In [5]:
Graph, node_names = read_graph(directed=False)

100%|██████████| 19576/19576 [00:00<00:00, 157723.19it/s]

Reading Nodes list


Reading Edges list


In [6]:
pipeline = Pipeline(Degree())
features = pipeline.apply(Graph, verbose=True)

degree_undirected


In [31]:
scores = features[:,0]
gene_query_Id = np.array(node_names.keys())[scores.argsort()[::-1]].tolist()
gene_query = np.array(node_names.values())[scores.argsort()[::-1]].tolist()
gene_rank = scores.copy()
gene_rank.sort()
gene_rank = gene_rank[::-1].tolist()

In [32]:
gene_ref_formatted = [gene[4:] for gene in cancer['gene_string']]
gene_query_formatted = [gene[9:] for gene in gene_query]

In [36]:
len(np.unique(gene_ref_formatted))

658

In [51]:
N = len(gene_query_formatted)
n = 100
M = len(list(set(gene_query_formatted) & set(gene_ref_formatted)))
m = len(list(set(gene_query_formatted[0:n]) & set(gene_ref_formatted)))
print N,n,M,m

19576 100 0 0


In [39]:
[gene for gene in gene_query_formatted if gene in gene_ref_formatted]

[]

In [50]:
len(gene_query_formatted[0:n])

100

## Enrichr: http://amp.pharm.mssm.edu/Enrichr/#

In [ ]:
gene_list = ['CTLA2B', 'SCARA3', 'LOC100044683', 'CMBL', 'CLIC6', 'IL13RA1', 'TACSTD2', 'DKKL1', 'CSF1', 'CITED1']
gene_rank = 3*np.random.random(len(gene_list))
rnk = pd.DataFrame(np.array([gene_list, gene_rank]).T, columns = ['gene', 'score'])
rnk

#rnk = pd.DataFrame(np.array([gene_list]).T)
#rnk

In [ ]:
enr = gp.enrichr(gene_list=gene_list, description='pathway', gene_sets='KEGG_2016', outdir='test', cutoff=0.05, format='png')
#enr = gp.enrichr(gene_list=gene_list, description='pathway', gene_sets=u'ARCHS4_Cell-lines', outdir='test', cutoff=0.05, format='png')
enr.res2d[enr.res2d["Adjusted P-value"]<0.1]

## Prerank: http://software.broadinstitute.org/cancer/software/genepattern/modules/docs/GSEAPreranked/1

In [ ]:
pre_res = gp.prerank(rnk=rnk, gene_sets='KEGG_2016', outdir='prerank_report',format='png')
pre_res.res2d.head()

## Cancer gene Census (Controllability Paper)

In [ ]:
cancer_gene_census = pd.read_csv("cancer_gene_census.csv")
gene_symbols = cancer_gene_census.loc[:,"Gene Symbol"]
gene_names = cancer_gene_census.loc[:,"Name"]
gene_string = []
list_of_synonyms = cancer_gene_census.loc[:,"Synonyms"]
for synonyms in list_of_synonyms:
    if str(synonyms) != 'nan':        
        for synonym in synonyms.split(','):
            if synonym[:3] == "ENS":
                gene_string.append(synonym)
print "Number of genes = ", len(gene_symbols)

In [ ]:
#with open("mimTitles.txt", "r") as f:
gene_entrez = []
gene_symbols = []
gene_string = []
with open("validation_datasets/mim2gene.txt") as f:
    for line in f:
        if len(line.split('\t'))>1:
            print line
            if line.split('\t')[1] == "gene":
                if len(line.split('\t'))>2:
                    gene_entrez.append(line.split('\t')[2])
                    gene_symbols.append(line.split('\t')[3])
                    gene_string.append(line.split('\t')[4])

print "Nb of genes =", len(gene_string)

In [ ]:
#subset: "all" or "approved"
#molecule_type: "carrier", "enzyme", "target", "transporter"
def get_drug_bank(subset="all", molecule_type="target"):
    data = pandas.read_csv("validation_datasets/drugbank_%s_%s_polypeptide_ids.csv/all.csv"%(subset, molecule_type))
    data = data[data["Species"]=="Human"]
    gene_names = data["Gene Name"].values.tolist()
    protein_names = data["Name"].values.tolist()
    uniprot_ID = data["UniProt ID"].values.tolist()
    return gene_names, protein_names, uniprot_ID

gene_names, protein_names, uniprot_ID = get_drug_bank(subset="all", molecule_type="transporter")